In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
import re
import string
from collections import Counter
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import display
from sklearn.preprocessing import LabelEncoder

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [46]:
file_path_train = '../data/train_preprocess.tsv'
train_data = pd.read_csv(file_path_train, sep='\t', header=None)

file_path_val = '../data/valid_preprocess.tsv'
val_data = pd.read_csv(file_path_val, sep='\t', header=None)

file_path_test = '../data/test_preprocess_masked_label.tsv'
test_data = pd.read_csv(file_path_test, sep='\t', header=None)

# Data Understanding
print('Data Train')
print('Data Size:', train_data.shape)
print('Sample Data:')
display(train_data.head())
print('\nSentiment Distribution:', train_data[1].value_counts())

print('\nData Validation')
print('Data Size:', val_data.shape)
print('Sample Data:')
display(val_data.head())

print('\nData Test')
print('Data Size:', test_data.shape)
print('Sample Data:')
display(test_data.head())

Data Train
Data Size: (11000, 2)
Sample Data:


,0,1
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative



Sentiment Distribution: 1
positive    6416
negative    3436
neutral     1148
Name: count, dtype: int64

Data Validation
Data Size: (1260, 2)
Sample Data:


,0,1
0,"meski masa kampanye sudah selesai , bukan bera...",neutral
1,tidak enak,negative
2,restoran ini menawarkan makanan sunda . kami m...,positive
3,lokasi di alun alun masakan padang ini cukup t...,positive
4,betapa bejad kader gerindra yang anggota dprd ...,negative



Data Test
Data Size: (500, 2)
Sample Data:


,0,1
0,kemarin gue datang ke tempat makan baru yang a...,neutral
1,kayak nya sih gue tidak akan mau balik lagi ke...,neutral
2,"kalau dipikir-pikir , sebenarnya tidak ada yan...",neutral
3,ini pertama kalinya gua ke bank buat ngurusin ...,neutral
4,waktu sampai dengan gue pernah disuruh ibu lat...,neutral


In [49]:
# Label Splitting
x_train = train_data[0]
y_train = train_data[1]

print('x_train:\n')
display(x_train.head())
print('y_train:\n')
display(y_train.head())

x_val = val_data[0]
y_val = val_data[1]
print('x_val:\n')
display(x_val.head())
print('y_val:\n')
display(y_val.head())

x_test = test_data[0]
y_test = test_data[1]
print('x_test:\n')
display(x_test.head())
print('y_test:\n')
display(y_test.head())

x_train:



0    warung ini dimiliki oleh pengusaha pabrik tahu...
1    mohon ulama lurus dan k212 mmbri hujjah partai...
2    lokasi strategis di jalan sumatera bandung . t...
3    betapa bahagia nya diri ini saat unboxing pake...
4    duh . jadi mahasiswa jangan sombong dong . kas...
Name: 0, dtype: object

y_train:



0    positive
1     neutral
2    positive
3    positive
4    negative
Name: 1, dtype: object

x_val:



...

0     neutral
1    negative
2    positive
3    positive
4    negative
Name: 1, dtype: object

x_test:



0    kemarin gue datang ke tempat makan baru yang a...
1    kayak nya sih gue tidak akan mau balik lagi ke...
2    kalau dipikir-pikir , sebenarnya tidak ada yan...
3    ini pertama kalinya gua ke bank buat ngurusin ...
4    waktu sampai dengan gue pernah disuruh ibu lat...
Name: 0, dtype: object

y_test:



0    neutral
1    neutral
2    neutral
3    neutral
4    neutral
Name: 1, dtype: object

In [51]:
# Label Encoding
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

print('Encoded labels train:', y_train_encoded)
print('Encoded labels validation:', y_val_encoded)
print('Classes:', label_encoder.classes_)

Encoded labels train: [2 1 2 ... 1 0 2]
Encoded labels validation: [1 0 2 ... 0 0 2]
Classes: ['negative' 'neutral' 'positive']


In [38]:
# Lowercase
x_train = x_train.str.lower()

print('Lowercase Data Train:')
display(x_train)

x_test = x_test.str.lower()
print('Lowercase Data Test:')
display(x_test)

Lowercase Data Train:


0        warung ini dimiliki oleh pengusaha pabrik tahu...
1        mohon ulama lurus dan k212 mmbri hujjah partai...
2        lokasi strategis di jalan sumatera bandung . t...
3        betapa bahagia nya diri ini saat unboxing pake...
4        duh . jadi mahasiswa jangan sombong dong . kas...
                               ...                        
10995                                         tidak kecewa
10996    enak rasa masakan nya apalagi kepiting yang me...
10997          hormati partai-partai yang telah berkoalisi
10998    pagi pagi di tol pasteur sudah macet parah , b...
10999    meskipun sering belanja ke yogya di riau junct...
Name: 0, Length: 11000, dtype: object

Lowercase Data Test:


0      kemarin gue datang ke tempat makan baru yang a...
1      kayak nya sih gue tidak akan mau balik lagi ke...
2      kalau dipikir-pikir , sebenarnya tidak ada yan...
3      ini pertama kalinya gua ke bank buat ngurusin ...
4      waktu sampai dengan gue pernah disuruh ibu lat...
                             ...                        
495    kata nya tidur yang baik itu minimal enam jam ...
496                    indonesia itu ada di benua asia .
497    salah satu kegemaran anak remaja indonesia sek...
498    melihat warna hijau bisa bikin mata jadi lebih...
499    bondan winarno yang suka bilang maknyus sekara...
Name: 0, Length: 500, dtype: object

In [39]:
# Entity Masking
# Email Masking
email = re.compile(r'\w+@\w+\.[a-z]{3}')
emails_found = x_train.apply(lambda x: email.findall(x))
print('Emails Found:')
display(emails_found)

x_train_mask = x_train.apply(lambda x: email.sub('_email_', x))
print('Email Masked:')
display(x_train_mask)

Emails Found:


0        []
1        []
2        []
3        []
4        []
         ..
10995    []
10996    []
10997    []
10998    []
10999    []
Name: 0, Length: 11000, dtype: object

Email Masked:


0        warung ini dimiliki oleh pengusaha pabrik tahu...
1        mohon ulama lurus dan k212 mmbri hujjah partai...
2        lokasi strategis di jalan sumatera bandung . t...
3        betapa bahagia nya diri ini saat unboxing pake...
4        duh . jadi mahasiswa jangan sombong dong . kas...
                               ...                        
10995                                         tidak kecewa
10996    enak rasa masakan nya apalagi kepiting yang me...
10997          hormati partai-partai yang telah berkoalisi
10998    pagi pagi di tol pasteur sudah macet parah , b...
10999    meskipun sering belanja ke yogya di riau junct...
Name: 0, Length: 11000, dtype: object

In [40]:
# Phone Number Masking
phone = re.compile(r'(\+?\d{1,2}[-.\s]?)?(\(?\d{3}\)?[-.\s]?)?\d{3}[-.\s]?\d{4}')
phones_found = x_train_mask.apply(lambda x: phone.findall(x))
print('Phones Found:')
print(phones_found)

print()
x_train_mask = x_train_mask.apply(lambda x: phone.sub('_phone_', x))
print('Phone Masked:')
print(x_train_mask)

Phones Found:
0        []
1        []
2        []
3        []
4        []
         ..
10995    []
10996    []
10997    []
10998    []
10999    []
Name: 0, Length: 11000, dtype: object

Phone Masked:
0        warung ini dimiliki oleh pengusaha pabrik tahu...
1        mohon ulama lurus dan k212 mmbri hujjah partai...
2        lokasi strategis di jalan sumatera bandung . t...
3        betapa bahagia nya diri ini saat unboxing pake...
4        duh . jadi mahasiswa jangan sombong dong . kas...
                               ...                        
10995                                         tidak kecewa
10996    enak rasa masakan nya apalagi kepiting yang me...
10997          hormati partai-partai yang telah berkoalisi
10998    pagi pagi di tol pasteur sudah macet parah , b...
10999    meskipun sering belanja ke yogya di riau junct...
Name: 0, Length: 11000, dtype: object


In [41]:
x_train_tokenized = x_train_mask.apply(lambda x: word_tokenize(x))
print('Tokenization:')
print(x_train_tokenized)

Tokenization:
0        [warung, ini, dimiliki, oleh, pengusaha, pabri...
1        [mohon, ulama, lurus, dan, k212, mmbri, hujjah...
2        [lokasi, strategis, di, jalan, sumatera, bandu...
3        [betapa, bahagia, nya, diri, ini, saat, unboxi...
4        [duh, ., jadi, mahasiswa, jangan, sombong, don...
                               ...                        
10995                                      [tidak, kecewa]
10996    [enak, rasa, masakan, nya, apalagi, kepiting, ...
10997    [hormati, partai-partai, yang, telah, berkoalisi]
10998    [pagi, pagi, di, tol, pasteur, sudah, macet, p...
10999    [meskipun, sering, belanja, ke, yogya, di, ria...
Name: 0, Length: 11000, dtype: object


In [43]:
# Stemming and Spelling Correction
factory = StemmerFactory()
stemmer = factory.create_stemmer()

x_train_stem = []
for sentence in x_train_tokenized:
    stemmed_words = [stemmer.stem(word) for word in sentence]
    x_train_stem.append(stemmed_words)

print('Stemming:')
x_train_stem

KeyboardInterrupt: 

In [ ]:
# Punctuation Removal
x_train_nopunc = []

for sentence in x_train_stem:
    cleaned_sentence = [word for word in sentence if word not in string.punctuation]
    x_train_nopunc.append(cleaned_sentence)

print('No punctuation:')
x_train_nopunc

In [ ]:
# Vectorization
x_train_cleaned = [' '.join(sentence) for sentence in x_train_nopunc]
tfidf_vec = TfidfVectorizer(min_df=100, token_pattern=r'[a-zA-Z]+')
x_train_bow = tfidf_vec.fit_transform(x_train_cleaned)

print(x_train_bow.shape)
print(x_train_bow.toarray())

In [ ]:
# 